In [ ]:
import sklearn

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

np.random.seed(2022)

def load_customer():
    # read the customers from the storage
    customers = pd.read_csv("customers.csv")
    # rename the column names with the CUSTOMER prefix
    new_column_names = {
        "REV_CURRENT_YEAR.1": "CUSTOMER_REV_CURRENT_YEAR",
        "REV_CURRENT_YEAR.2": "CUSTOMER_REV_PAST_YEAR",
        "CREATION_YEAR": "CUSTOMER_CREATION_YEAR",
        "OWNERSHIP": "CUSTOMER_OWNERSHIP",
        "CURRENCY": "CUSTOMER_CURRENCY"
    }
    customers.rename(columns=new_column_names, inplace=True)
    # map the country to the same format in the geo csv
    customers['COUNTRY'] = customers['COUNTRY'].map({'Switzerland': 'CH', 'France': 'FR'})
    # this column has no information because it's the same then CUSTOMER_REV_CURRENT_YEAR
    customers.drop(columns=['REV_CURRENT_YEAR'], inplace=True)
    return customers


def load_geo():
    geo = pd.read_csv('geo.csv')
    new_column_names = {
        "SALES_OFFICE": "GEO_SALES_OFFICE",
        "SALES_BRANCH": "GEO_SALES_BRANCH",
    }
    geo.rename(columns=new_column_names, inplace=True)
    return geo


def load_transaction():
    transactions = pd.read_csv("transactions.csv")
    transactions['CUSTOMER'] = transactions['CUSTOMER'].str.replace('"', '')
    transactions['CUSTOMER'] = transactions['CUSTOMER'].replace('#NV', np.nan)
    transactions['CUSTOMER'] = pd.to_numeric(transactions['CUSTOMER'], errors='coerce')
    # map the offer status to a boolean
    transactions['OFFER_STATUS'] = transactions['OFFER_STATUS'].apply(
        lambda x: 1 if x in ['WIN', 'Win', 'WON', 'Won'] else 0)
    return transactions


def load_data():
    transactions_geo = pd.merge(left=load_transaction(), right=load_geo(), on='SALES_LOCATION', how='left')
    transactions_geo_customer = pd.merge(left=transactions_geo, right=load_customer(), on=['CUSTOMER', "COUNTRY"],
                                         how='left')
    transactions_geo_customer.columns = transactions_geo_customer.columns.str.lower()
    return transactions_geo_customer

In [ ]:
from sklearn.model_selection import train_test_split

def split_validation_test_train(df):
    validation = df[~pd.isna(df['test_set_id'])]
    _X_val = validation.drop(['offer_status', 'test_set_id'], axis=1)
    _y_test_set_id = validation['test_set_id']
    data = df[pd.isna(df['test_set_id'])].drop('test_set_id', axis=1)
    X = data.drop('offer_status', axis=1)
    y = data['offer_status']
    _X_train, _X_test, _y_train, _y_test = train_test_split(X, y, test_size=0.2, random_state=2022)
    return _X_val, _X_train, _X_test, _y_test_set_id, _y_train, _y_test

In [ ]:
X_val, X_train, X_test, y_test_set_id, y_train, y_test = split_validation_test_train(load_data())

In [ ]:
Exploratory = X_train.copy()

In [ ]:
def feature_engineering(df):
    # CUSTOMER
    # map the different customer currency to euro
    to_Euro = {'Pound Sterling': 1.19, 'US Dollar': 0.88, 'Chinese Yuan': 0.14, 'Euro': 1}
    df["currency_factor"] = df["customer_currency"].map(to_Euro)
    df["customer_rev_current_year"] = df["customer_rev_current_year"] * df["currency_factor"]
    df["customer_rev_past_year"] = df["customer_rev_past_year"] * df["currency_factor"]
    # The delta between the revenue of the last and the current year
    df["customer_rev_increase"] = df["customer_rev_current_year"] - df["customer_rev_past_year"]
    # Calculate the distance between the current year and when the customer comes.
    df['customer_creation_year'] = pd.to_datetime(df['customer_creation_year'])
    df['customer_creation_year'] = 2022 - df['customer_creation_year'].apply(lambda x: x.year)

    # TRANSACTION
    df['end_customer'] = df['end_customer'].fillna('No').apply(lambda x: x if x in ['No', 'Yes'] else 'Yes')
    offers_count = df['mo_id'].value_counts()
    df['offers_count'] = df['mo_id'].apply(lambda x: offers_count[x])
    # The columns to drop
    df = df.drop(columns=['currency_factor', 'mo_id', 'so_id', 'customer', 'mo_created_date', 'so_created_date', 'costs_product_a', 'costs_product_b', 'costs_product_c', 'costs_product_d', 'costs_product_e', 'customer_currency'], axis=1)
    #
    df = df.drop(columns=['sales_location', 'country', 'geo_sales_office', 'geo_sales_branch', 'customer_rev_current_year', 'customer_rev_past_year','customer_ownership', 'customer_creation_year', 'material_cost', 'service_cost', 'price_list', 'isic'], axis=1)
    return df

Exploratory = feature_engineering(X_train)
Exploratory

In [ ]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

numeric_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='mean'))
      ,('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='constant'))
      ,('encoder', OrdinalEncoder())
])

numeric_features = [col for col in Exploratory.columns if Exploratory.dtypes[col] != 'object']
categorical_features = [col for col in Exploratory.columns if Exploratory.dtypes[col] == 'object']
print(numeric_features, categorical_features)
preprocessor = ColumnTransformer(
   transformers=[
    ('numeric', numeric_transformer, numeric_features)
   ,('categorical', categorical_transformer, categorical_features)
])

In [ ]:
from sklearn.linear_model import LogisticRegression
pipeline = Pipeline(steps = [
               ('preprocessor', preprocessor)
              ,('regressor',LogisticRegression())
           ])
rf_model = pipeline.fit(Exploratory, y_train)

In [ ]:
from sklearn.metrics import balanced_accuracy_score
y_pred = rf_model.predict(feature_engineering(Exploratory))
print (balanced_accuracy_score(y_test, y_pred))

In [ ]:
test = feature_engineering(Exploratory)

In [ ]:
[col for col in test.columns if test.dtypes[col] == 'object']

In [ ]:
preprocessor.transform(prepared_X_test)